In [ ]:

import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import re
from sklearn.model_selection import train_test_split
import joblib


In [ ]:
ds = load_dataset('7Xan7der7/us_airline_sentiment')
df = ds['train'].to_pandas()
df.head()

In [ ]:
df['airline_sentiment'].value_counts()


In [ ]:
df = df[['text','airline_sentiment']]
df.describe(include='all')

In [ ]:
df['airline_sentiment'].nunique()

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        return ''
    text = re.sub(r'http+', '', text)
    text = re.sub(r'@+', '', text)
    text = re.sub(r'[^A-Za-z0-9]', ' ', text)
    text = text.lower().strip()
    return text

df['text'] = df['text'].apply(clean_text)
df = df.drop_duplicates(subset=['text'])
df = df.dropna(subset=['text'])
df.to_csv('./data/processed/batch_processed.csv', index=False)

df.describe(include='all')
